# 1. Import modules/libraries

In [ ]:
import ee, geemap, os, calendar, datetime, shapefile, sys, glob

import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt

In [ ]:
Map = geemap.Map(center=(10.393211,37.377579), zoom=12)
Map

## Define Season

In [ ]:
start = '2021-12-01'
end = '2022-04-31'
season = ee.Filter.date(start,end)

# 2. Import the input data:

## 2.1. Read SEBALI Outputs from Google Drive

## 2.2. Read geographical extent of the study area

In [13]:
shumbrit = ee.FeatureCollection("projects/ee-yilkalgebeyehu/assets/Shumbrit_Bounday")
Map.addLayer(shumbrit, {}, 'Scheme')

# 3. Caculate Seasonal Parameters

## 3.1. Define and import the Kc per month

From CROPWAT 8 software

## 3.2. Seasonal Rainfall

In [ ]:
# Monthly Rainfall from CHIRPS 
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filter(season)\
    .sum()


rain = chirps.reduceRegion(**{
  'reducer': ee.Reducer.mean(),
  'geometry': shumbrit.geometry(),
  'scale': 5566,
  'maxPixels': 1e9
})

# The result is a Dictionary.  Print it.
rain = rain.get('precipitation')
print('Seasonal Rainfall (mm)',  rain.getInfo())

# 4. Performance Indicators

## 4.1. Calculate uniformity of water consumption
* Equity is defined as the coefficients of variation (CV) of seasonal ETa in the area of interest.
* It measures the evenness of the water supply in an irrigation scheme.
* Note: CV of 0 to 10% is good, 10 to 25% is fair and CV > 25% is poor uniformity (Bastiaanssen et al., 1996)

In [ ]:
# Uniformity of water Consumption 
# get seasonal AETI
# AETI_fhs = glob.glob(r'..\Data\2L2_AETI_season\*.tif') 

for i in range(len(AETI_fhs)):
    AETI = gis.OpenAsArray(AETI_fhs[i], nan_values=True)
    
    AETIm   = np.nanmean(AETI)
    AETIsd  = np.nanstd(AETI)
    
    CV_AETI = (AETIsd / AETIm) * 100
    
    # Identify the date from the file name
    date  = os.path.basename(AETI_fhs[i]).split('.')[0].replace('AETI', '').replace('_', ' ') 
    
    if CV_AETI < 10:
        U = 'Good Uniformity'
    if (CV_AETI >= 10) and (CV_AETI < 25):
        U = 'Fair Uniformity'
    else: 
        U = 'Poor Uniformity'

    print ('CV of AETI in', date, '=', round(CV_AETI, 1), ',', U)

## 4.2. Calculate adequacy (relative evapotranspiration)

In [ ]:
## Calculate and save raster adequacy layer

# collecting Geoinfo such as projection, the x and y axis
in_fh = AETI_fhs[0]      
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # get spatial extent of raster

for ETfh, ETpfh in zip(AETI_fhs, ETp_fhs):
    AETI = gis.OpenAsArray(ETfh,  nan_values=True)
    ETp  = gis.OpenAsArray(ETpfh, nan_values=True)
    
    ETa_by_ETp = AETI / ETp
    
    # Identify the date from the file name
    date  = os.path.basename(ETfh).split('.')[0].replace('AETI', '').replace('_', ' ') 
        
    # update the file name, and save into output folder
    basename  = os.path.basename(ETfh).replace('AETI', 'Adequacy')
    output_fn = os.path.join(output_folderAd, basename)
    gis.CreateGeoTiff(output_fn, ETa_by_ETp, driver, NDV, xsize, ysize, GeoT, Projection) 
  
    # Plot the raster map
    seasonal = ETa_by_ETp
    
    plt.figure(figsize = (12,8))
    plt.imshow(seasonal, cmap='RdYlGn', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    plt.colorbar(shrink=0.75, label='Adequacy [-]')
    plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=12)  # add axes label
    plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=12)
    plt.title('Adequacy [-] ' + date)
    plt.show ()

## 4.3. Calculate Relative water Deficit (RWD)

RWD = 1-ETa/ETx

In [ ]:
# collecting Geoinfo such as projection, the x and y axis
in_fh = AETI_fhs[0]      
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # get spatial extent of raster
   
for i in range(len(AETI_fhs)):
    AETI = gis.OpenAsArray(AETI_fhs[i], nan_values=True)
    
    # reshape the arrays
    AETI1_1D  = np.reshape(AETI,  AETI.shape[0]*AETI.shape[1])
    ETx       = np.nanpercentile(AETI1_1D, 99)
    
    AETI_mean = np.nanmean(AETI)
   
    RWD = 1-(AETI_mean/ETx)
    
    # Identify the date from the file name
    date  = os.path.basename(AETI_fhs[i]).split('.')[0].replace('AETI', '').replace('_', ' ') 
    
    print ('Relative water deficit', date, '=', round(RWD, 2))

## 4.4. Depleted Fraction

DF = ETa/Rain+Vc
* ETa is actual evapotranspiration
* Rain: seasonal rainfall calculated from CHIRPS daily preciptation
* Vc is the volume of irrigation supply 

In [ ]:
df = ETa_season.devide(rain).add(vc)

## 4.5. Crop Water Productivity

CWP = Yield/ETa

## 4.6. Sustainablity

Based on Irrigated area